In [90]:
# from google.colab import drive
# drive.mount('/content/drive')

In [91]:
import pickle
import numpy as np

with open('raw_data.pkl', 'rb') as f:
    data = pickle.load(f)
    
    
from sklearn.model_selection import train_test_split
X_train, x_test, y_train, y_test = train_test_split(np.array(data[0]),np.array(data[1]),stratify=data[1],test_size=0.2,random_state=777,shuffle=True)


    


In [92]:
print(X_train.shape)
print(x_test.shape)
print(len(X_train),len(X_train[1]))
print(len(x_test),len(x_test[1]))
print(y_test.shape)
print(y_train.shape)

(7680, 11025)
(1920, 11025)
7680 11025
1920 11025
(1920,)
(7680,)


In [93]:
print(len(np.unique(y_train)))
print(len(np.unique(y_test)))

600
600


In [94]:
import pandas as pd 


In [95]:
# import tensorflow as tf
# tf.version.VERSION

# # df = pd.DataFrame(data=x_train)
# # print(df)
# #[7680 rows x 11025 columns]


In [96]:

#basic imports
import glob
import IPython
from random import randint

#data processing

import numpy as np

#modelling
from sklearn.model_selection import train_test_split

from keras import backend as K
from keras.layers import Activation
from keras.layers import Input, Lambda, Dense, Dropout, Flatten
from keras.models import Model
from keras.optimizers import RMSprop



In [149]:
# https://www.pyimagesearch.com/2020/11/23/building-image-pairs-for-siamese-networks-with-python/
import random
def make_pairs(images, labels):
    pairImages = []
    pairLabels = []
    numClasses = len(np.unique(labels))
    
    #idx = [np.where(labels == i)[0] for i in range(0, numClasses)]
    
#     print(len(labels))
   
    idx = []
    counter = 0
    for i in range(0, max(labels)+1):
        personPoints = (np.where(labels == i)[0])
        if len(personPoints) > 0: 
            
#             randomValue = random.randint(0, len(personPoints)-1)
#             print(personPoints)
            idx.append(personPoints)
#             print(idx)
            
        else:
            #determine which labels are not used
            #print(i)
            
            counter+=1
            #print(counter)
            idx.append(False)
       
    
    #print(len(idx))
  #for i in range(0, max(labels)+1):
  #  idxs = np.where(labels == i)[0]
  #  print("{}: {} {}".format(i, len(idxs), idxs))
    for idxA in range(0, max(labels)+1):
        #grab current audioclip and label
        currentImage = images[idxA]
        label = labels[idxA]

#         if (idx[label] == 0):
#             print(idx[label])
        #randomly pick clip with same label
        idxB = np.random.choice(idx[label])
        posImage = images[idxB]
        
        #prepare a positive pauir and update audio and labels
        pairImages.append([currentImage, posImage])
        pairLabels.append([1])

        #create negative pairs
        negIdx = np.where(labels != label)[0]
        
        
        randomValue = random.randint(0, len(negIdx)-1)
        negId = negIdx[randomValue]
        
        #Why would we choose a random 'image' here? We want to keep the labels te same as the actual values right?
        negImage = images[negId]
        
        #prepare a negative pair and update lists
        pairImages.append([currentImage, negImage])
        pairLabels.append([0])
        
#         if (y_train[idxA] == y_train[negId]):
#             # throw when idxA and negIdx are the same person
#             print("ERRRORRRRRR")
#         print(len(pairImages))
    return (np.array(pairImages), np.array(pairLabels))

(pairTrain, labelTrain) = make_pairs(X_train, y_train)

In [151]:
# (pairTrain, labelTrain) = make_pairs(X_train, y_train)
numClasses = len(np.unique(y_train))
# print(numClasses)

# print(pairTrain.shape)
# print(pairTrain[0])


print(pairTrain.shape)
print(labelTrain.shape)
with open('test.txt', 'w') as f:
    for item in pairTrain:
         f.write("%s\n" % item)

# for pair in pairTrain:
#     for value in pair: 
#         if (value[0] == 0):
#             print(value[0])
# print(y_train[1])
# print(y_train[2762])

X_train = pairTrain
# print(y_train[1013])
# print(y_train[2417])

(2028, 2, 11025)
(2028, 1)


In [152]:
def build_base_network(input_shape):
    input = Input(shape=input_shape)
    x = Flatten()(input)
    x = Dense(128, activation='relu')(x)
    x = Dropout(0.1)(x)
    x = Dense(128, activation='relu')(x)
    x = Dropout(0.1)(x)
    x = Dense(128, activation='relu')(x)
    return Model(input, x)

In [153]:

input_dim = X_train.shape[2:]

audio_a = Input(shape=input_dim)
audio_b = Input(shape=input_dim)

In [154]:
base_network = build_base_network(input_dim)

feat_vecs_a = base_network(audio_a)
feat_vecs_b = base_network(audio_b)

In [155]:
def euclidean_distance(vects):
    x, y = vects
    return K.sqrt(K.sum(K.square(x - y), axis=1, keepdims=True))


def eucl_dist_output_shape(shapes):
    shape1, shape2 = shapes
    return (shape1[0], 1)

In [156]:
distance = Lambda(euclidean_distance, output_shape=eucl_dist_output_shape)([feat_vecs_a, feat_vecs_b])

In [157]:
epochs = 13
rms = RMSprop()

In [158]:
model = Model([audio_a, audio_b], distance)

In [159]:
def contrastive_loss(y_true, y_pred):
    margin = 1
    return K.mean(y_true * K.square(y_pred) + (1 - y_true) * K.square(K.maximum(margin - y_pred, 0)))

In [160]:
model.compile(loss=contrastive_loss, optimizer=rms)

In [161]:
audio_1 = X_train[:, 0]
audio_2 = X_train[:, 1]
print(len(audio_2))
print(len(X_train))

print(audio_2)
print(X_train)

2028
2028
[[-0.07931909 -0.13752252 -0.23027724 ...  0.00196907  0.00444865
   0.01781415]
 [ 0.05252755 -0.05839225  0.02091636 ... -0.02272057  0.02656228
   0.03136083]
 [-0.01365361 -0.01069179 -0.01241004 ... -0.09604666 -0.08082318
  -0.05730995]
 ...
 [ 0.03885582  0.02049039  0.05103874 ...  0.00387909  0.00417048
   0.00097108]
 [-0.7252695  -0.69353575 -0.57106483 ... -0.08805424 -0.09067984
  -0.04239523]
 [-0.00517329  0.00188006  0.00836466 ...  0.18082255  0.18206796
   0.20113985]]
[[[ 7.06316263e-04 -2.36646198e-02 -1.29810805e-02 ... -6.82306886e-02
   -6.56690076e-02 -8.61486346e-02]
  [-7.93190897e-02 -1.37522519e-01 -2.30277240e-01 ...  1.96906738e-03
    4.44864715e-03  1.78141538e-02]]

 [[ 7.06316263e-04 -2.36646198e-02 -1.29810805e-02 ... -6.82306886e-02
   -6.56690076e-02 -8.61486346e-02]
  [ 5.25275543e-02 -5.83922528e-02  2.09163558e-02 ... -2.27205679e-02
    2.65622791e-02  3.13608274e-02]]

 [[-1.87887845e-03 -2.27281172e-03 -2.47887266e-03 ... -2.33109936

In [163]:

print(audio_1)
print(audio_2)
y_train = y_train.astype(np.float)
print(y_train)
print(type(y_train[0]))

model.fit([audio_1, audio_2], y_train, validation_split=.25,
          batch_size=128, verbose=2, epochs=epochs)
model.summary()

[[ 0.00070632 -0.02366462 -0.01298108 ... -0.06823069 -0.06566901
  -0.08614863]
 [ 0.00070632 -0.02366462 -0.01298108 ... -0.06823069 -0.06566901
  -0.08614863]
 [-0.00187888 -0.00227281 -0.00247887 ... -0.23310994 -0.22018412
  -0.21164122]
 ...
 [-0.02360282 -0.02762017 -0.02718573 ...  0.18859276 -0.27450654
   0.19044574]
 [-0.10945068  0.04494718  0.00506724 ... -0.35359558  0.32809448
  -0.14303342]
 [-0.10945068  0.04494718  0.00506724 ... -0.35359558  0.32809448
  -0.14303342]]
[[-0.07931909 -0.13752252 -0.23027724 ...  0.00196907  0.00444865
   0.01781415]
 [ 0.05252755 -0.05839225  0.02091636 ... -0.02272057  0.02656228
   0.03136083]
 [-0.01365361 -0.01069179 -0.01241004 ... -0.09604666 -0.08082318
  -0.05730995]
 ...
 [ 0.03885582  0.02049039  0.05103874 ...  0.00387909  0.00417048
   0.00097108]
 [-0.7252695  -0.69353575 -0.57106483 ... -0.08805424 -0.09067984
  -0.04239523]
 [-0.00517329  0.00188006  0.00836466 ...  0.18082255  0.18206796
   0.20113985]]
[374. 548. 670. 